# Task:

- Implement batch gradient descent with early stopping for softmax regression without using Scikit-Learn, only NumPy. Use it on a classification task such as the iris dataset.

## Softmax Regression

Generalization of *Logistic Regression* to support multiple classes. 

Given an instance `x`, the model computes a score $s_k(x)$ for each class `k`, then estimates the probability of each class applying the *softmax function* (normalized exponential) to the scores.

The equation to compute $s_k(x)$ is:

$$
s_k(x) = \left( \theta^{(k)} \right)^T  x
$$

Where:
- $s_k(x)$ is the score of class `k` for instance `x`
- $\Theta$ is the parameter matrix where each row represents the parameter vector of class `k`

Once we have every class score (also called *logits* or *log-odds*) for the instance `x`, we estimate the porbability $\hat{p}_k$ of the instance belonging to class `k` using the softmax function:

$$
\hat{p}_k = \sigma(s(x))_k = \frac{ \exp(s_k(x)) }{ \sum_{j=1}^{K} \exp(s_j(x)) }
$$

Where:
- $K$ is the number of classes
- $s(x)$ is a vector containing the scores of each class for the instance `x`
- $\sigma(s(x))_k$ is the estimated probability that the instance `x` belongs to class `k`

Now, the prediction $\hat{y}$ is the class with the highest estimated probability:

$$
\hat{y} = \underset{k}{\text{argmax}} \, \sigma(s(x))_k = \underset{k}{\text{argmax}} \, s_k(x) = \underset{k}{\text{argmax}} \, \left( \theta^{(k)} \right)^T  x
$$

## Analysis of the problem

**Using only NumPy**

1. Load the iris dataset
1. Divide into train, test and validation sets
1. Scale the data
1. Implement the softmax regression model
1. Implement the batch gradient descent algorithm
1. Implement early stopping
1. Train the model
1. Evaluate the model

In [1]:
import numpy as np

# Scikit-learn just to load the data
from sklearn.datasets import load_iris

In [2]:
import sys
import os
# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'MyClasses')))

# Import the class from the other directory
import ProcessData as pdata

In [3]:
iris= load_iris()

In [4]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

In [5]:
# Split the data into features and labels
X = iris.data
y = iris.target

In [6]:
# Process using my class
def preprocess(X, y):
	pd = pdata.ProcessData()

	# Scale the features (normalize)
	X_normalized = pd.normalize(X)

	# The dummy feature is a column of ones to account for the bias term
	# Done *after* scaling
	X_norm_dummy = pd.add_dummy(X_normalized)

	# One-hot encode the labels
	# Labels are a probaility 1 for their class, 0 for the others 
	y_one_hot = pd.one_hot_encoder(y)

	return X_norm_dummy, y_one_hot

In [7]:
def process_split(X, y, test_ratio=0.2, val_ratio=0.2, random_state=42):
	X_processed, y_processed = preprocess(X, y)
	pd = pdata.ProcessData()

	X_train, y_train, X_val, y_val, X_test, y_test = pd.split_data(
		X_processed, y_processed,
		test_ratio=test_ratio,
		val_ratio=val_ratio,
		random_state=random_state
		)
	
	return X_train, y_train, X_val, y_val, X_test, y_test

In [19]:
X_train, y_train, X_val, y_val, X_test, y_test = process_split(X, y)

## Softmax Function

$
\sigma(s(x))_k = \frac{ \exp(s_k(x)) }{ \sum_{j=1}^{K} \exp(s_j(x)) }
$

In [9]:
def softmax(logits):
	exp = np.exp(logits)
	exp_sum = np.sum(exp, axis=1, keepdims=True)
	return exp / exp_sum

In [20]:
n_inputs = X_train.shape[1]
n_outputs = np.unique(y).shape[0]
n_inputs, n_outputs

(5, 3)

In [34]:
def training_phase(X_train, y_train, X_val, y_val, eta, n_epochs, epsilon, alpha,
				   Theta=None, theta_path_param=0.1, n_prints=10, random_state=42):

	# Some parameters
	m = len(X_train)
	divisor = n_epochs // n_prints

	# Theta path
	theta_path = []

	# Initial state
	if Theta is None:
		np.random.seed(random_state)
		Theta = np.random.randn(n_inputs, n_outputs)

	# Training loop
	for epoch in range(n_epochs):

		# Register the thetas
		if epoch % int(n_epochs * theta_path_param) == 0:
			theta_path.append(Theta)
		
		logits = X_train.dot(Theta)
		Y_proba = softmax(logits)
		# Print the cross entropy loss every 1/n_prints% of the epochs
		if epoch % divisor == 0:
			Y_proba_val = softmax(X_val.dot(Theta))
			l2_loss = 1/2 * np.square(Theta[1:]).sum()
			cross_entropy_loss = -(Y_proba_val*np.log(Y_proba_val + epsilon)).sum(axis=1).mean()
			total_loss = cross_entropy_loss + alpha * l2_loss
			print(epoch, total_loss)

		error = Y_proba - y_train
		gradient = 1/m * X_train.T.dot(error)
		# L2 regularization
		gradient += np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]] 
		Theta = Theta - eta * gradient

	return Theta, theta_path

In [12]:
eta = 0.5
n_epochs = 1000001
epsilon = 1e-7


Theta, theta_path = training_phase(X_train, y_train, X_val, y_val, eta, n_epochs, epsilon)

0 0.479975814211785
100000 0.018557923360427105
200000 0.014264666944663964
300000 0.012280822049746414
400000 0.010954423221552292
500000 0.009941804902850956
600000 0.00911957399585182
700000 0.00842806494689668
800000 0.007832990821644252
900000 0.007312427965791503
1000000 0.006851331425348677


In [13]:
def predict(X, Theta):
	logits = X.dot(Theta)
	Y_proba = softmax(logits)
	y_pred = Y_proba.argmax(axis=1)
	return y_pred

In [16]:
def acurracy(y, y_pred):
	return np.mean(y == y_pred)

In [24]:
pd = pdata.ProcessData()
# Using the validation dataset
y_pred = predict(X_test, Theta)
y_test_decoded = pd.one_hot_decoder(y_test)
accurracy_score = acurracy(y_test_decoded, y_pred)

In [25]:
accurracy_score

0.9666666666666667

In [35]:
eta = 0.5
n_epochs = 1000001
epsilon = 1e-7
alpha = 0.01

Theta_reg, theta_path_reg = training_phase(X_train, y_train, X_val, y_val, eta, n_epochs, epsilon, alpha)

0 0.5457413310462084
100000 0.4099727321521387
200000 0.4099727321521387
300000 0.4099727321521387
400000 0.4099727321521387
500000 0.4099727321521387
600000 0.4099727321521387
700000 0.4099727321521387
800000 0.4099727321521387
900000 0.4099727321521387
1000000 0.4099727321521387


In [36]:
Theta_reg

array([[ 0.05437073,  2.14354377, -1.19177611],
       [-0.9572536 ,  0.51685915,  0.44039445],
       [ 1.02318294, -0.34643789, -0.67674506],
       [-1.72396977, -0.26025226,  1.98422203],
       [-1.61520643, -0.71803684,  2.33324326]])

In [40]:
y_pred_reg = predict(X_test, Theta_reg)
acurracy_score_reg = acurracy(y_test_decoded, y_pred_reg)

acurracy_score_reg

0.9666666666666667